## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-25-20-49-40 +0000,bbc,Ukraine says 'understanding' reached with US o...,https://www.bbc.com/news/articles/cy95jvw57v2o...
1,2025-11-25-20-47-00 +0000,wsj,The Centers for Disease Control and Prevention...,https://www.wsj.com/politics/policy/trump-cdc-...
2,2025-11-25-20-46-00 +0000,wsj,A group of Democratic lawmakers said the FBI h...,https://www.wsj.com/politics/policy/democratic...
3,2025-11-25-20-38-18 +0000,nyt,BBC Accused of Censoring Trump by Dutch Histor...,https://www.nytimes.com/2025/11/25/world/europ...
4,2025-11-25-20-37-46 +0000,nyt,Minority Alawites Protest in Syria After Secta...,https://www.nytimes.com/2025/11/25/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,48
0,ukraine,26
3,peace,23
168,new,19
4,plan,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
182,2025-11-25-12-56-44 +0000,nypost,"Trump Army secretary meets Russians after US, ...",https://nypost.com/2025/11/25/us-news/trump-ar...,140
0,2025-11-25-20-49-40 +0000,bbc,Ukraine says 'understanding' reached with US o...,https://www.bbc.com/news/articles/cy95jvw57v2o...,135
72,2025-11-25-18-50-55 +0000,nypost,How Army Secretary Dan Driscoll rose from Trum...,https://nypost.com/2025/11/25/us-news/how-army...,134
31,2025-11-25-20-03-41 +0000,nypost,Trump dispatches top envoys to Russia and Ukra...,https://nypost.com/2025/11/25/us-news/trump-sa...,134
287,2025-11-25-00-42-05 +0000,nyt,What to Know About Trump’s Peace Plan for Russ...,https://www.nytimes.com/2025/11/24/us/politics...,128


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
182,140,2025-11-25-12-56-44 +0000,nypost,"Trump Army secretary meets Russians after US, ...",https://nypost.com/2025/11/25/us-news/trump-ar...
262,54,2025-11-25-03-24-31 +0000,nypost,Initial Ukraine-Russia peace plan was hashed o...,https://nypost.com/2025/11/24/us-news/initial-...
216,50,2025-11-25-10-30-00 +0000,wsj,Would You Like a New Car With That $8 USB Cabl...,https://www.wsj.com/business/retail/would-you-...
49,48,2025-11-25-19-31-36 +0000,nypost,"Family of Bethany MaGee, woman set on fire by ...",https://nypost.com/2025/11/25/us-news/family-o...
147,44,2025-11-25-15-26-18 +0000,latimes,13-year-old girl arrested after fatally shooti...,https://www.latimes.com/california/story/2025-...
276,40,2025-11-25-02-00-00 +0000,wsj,Allies of Federal Reserve Chair Jerome Powell ...,https://www.wsj.com/economy/central-banking/fe...
57,40,2025-11-25-19-12-00 +0000,wsj,Speaker Mike Johnson cautioned the White House...,https://www.wsj.com/politics/policy/speaker-jo...
356,39,2025-11-24-21-11-16 +0000,bbc,Judge dismisses cases against ex-FBI director ...,https://www.bbc.com/news/articles/c9qelv51y23o...
278,38,2025-11-25-01-39-34 +0000,nypost,Majority of Hispanic Republican voters say pre...,https://nypost.com/2025/11/24/us-news/majority...
164,35,2025-11-25-14-22-37 +0000,cbc,Court order prevents L.A. coroner from releasi...,https://www.cbc.ca/news/entertainment/court-or...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
